# Clipper Colab Backend\n\nRuns your Clipper FastAPI backend on Colab and exposes a public URL (Cloudflare tunnel).\n\nUse the generated URL in the app header `API URL` field on your phone.

In [ ]:
REPO_URL = "https://github.com/REPLACE_WITH_YOUR_USERNAME/clipper.git"\nBRANCH = "main"\nWORKDIR = "/content/clipper"\nMODEL_PATH = "/content/AIModels"\n\n# Keep this small at first. Add "video" only when ready for long downloads.\nDOWNLOAD_MODELS = ["image", "inpaint"]\n\n# Set True to attempt CPU text-to-video if no GPU is available.\nALLOW_CPU_T2V = True

In [ ]:
import os\nimport subprocess\nimport sys\n\nif "REPLACE_WITH_YOUR_USERNAME" in REPO_URL:\n    raise ValueError("Set REPO_URL to your actual GitHub repo first.")\n\nif os.path.exists(WORKDIR):\n    subprocess.run(["rm", "-rf", WORKDIR], check=True)\n\nsubprocess.run(["git", "clone", "--depth", "1", "--branch", BRANCH, REPO_URL, WORKDIR], check=True)\nos.chdir(WORKDIR)\n\nsubprocess.run(["apt-get", "-y", "update"], check=True)\nsubprocess.run(["apt-get", "-y", "install", "ffmpeg"], check=True)\n\nsubprocess.run([sys.executable, "-m", "pip", "install", "-U", "pip"], check=True)\nsubprocess.run([sys.executable, "-m", "pip", "install", "-r", "backend/requirements.txt"], check=True)\nsubprocess.run([sys.executable, "-m", "pip", "install", "-r", "backend/requirements-v2.txt", "--extra-index-url", "https://abetlen.github.io/llama-cpp-python/whl/cpu"], check=True)\n\nprint("Setup complete")

In [ ]:
import subprocess\nimport sys\n\nif DOWNLOAD_MODELS:\n    subprocess.run([sys.executable, "scripts/download_real_models.py", "--model-path", MODEL_PATH, "--targets", *DOWNLOAD_MODELS], check=True)\n    print("Downloaded:", DOWNLOAD_MODELS)\nelse:\n    print("Skipping model download")

In [ ]:
import os\nimport subprocess\nimport sys\nimport time\nimport requests\n\nos.environ["CLIPPER_MODEL_PATH"] = MODEL_PATH\nos.environ["CLIPPER_ALLOW_CPU_T2V"] = "1" if ALLOW_CPU_T2V else "0"\nos.environ["CLIPPER_STRICT_REAL_IMAGE"] = "1"\nos.environ["CLIPPER_STRICT_REAL_INPAINT"] = "1"\n\nbackend = subprocess.Popen(\n    [sys.executable, "-m", "uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"],\n    cwd=f"{WORKDIR}/backend"\n)\n\nok = False\nfor _ in range(60):\n    try:\n        r = requests.get("http://127.0.0.1:8000/health", timeout=2)\n        if r.status_code == 200:\n            ok = True\n            break\n    except Exception:\n        pass\n    time.sleep(1)\n\nif not ok:\n    raise RuntimeError("Backend did not become healthy")\n\nprint("Backend running on :8000")

In [ ]:
import os\nimport re\nimport subprocess\nimport time\n\nCF_BIN = "/content/cloudflared"\nif not os.path.exists(CF_BIN):\n    subprocess.run(["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-O", CF_BIN], check=True)\n    subprocess.run(["chmod", "+x", CF_BIN], check=True)\n\ntunnel = subprocess.Popen(\n    [CF_BIN, "tunnel", "--url", "http://127.0.0.1:8000", "--no-autoupdate"],\n    stdout=subprocess.PIPE,\n    stderr=subprocess.STDOUT,\n    text=True,\n    bufsize=1\n)\n\npublic_url = None\ndeadline = time.time() + 90\npattern = re.compile(r"https://[-a-z0-9]+\\.trycloudflare\\.com")\n\nwhile time.time() < deadline:\n    line = tunnel.stdout.readline()\n    if not line:\n        time.sleep(0.2)\n        continue\n    match = pattern.search(line)\n    if match:\n        public_url = match.group(0)\n        break\n\nif not public_url:\n    raise RuntimeError("Tunnel URL not found. Re-run this cell.")\n\nprint("PUBLIC_API_URL:", public_url)\nprint("Paste this into app header API URL field on phone")

## Use From Phone\n\n1. Open your frontend URL on phone.\n2. Paste `PUBLIC_API_URL` into `API URL` at the top of the app.\n3. Click `Use API`.\n\nTip: Colab sessions can stop. If it disconnects, rerun backend+tunnel cells.